In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import cv2
import gc


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import tensorflow as tf
import numpy as np
import random  # Import the random module

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)


In [6]:
df = pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/train.csv')
df.head()
df = df.sort_values("target", ascending = False)

In [7]:
df_1 = df.iloc[:584]
df_0 = df.iloc[584:1440]
df_0 = df_0.sample(frac=1, random_state=42)  # Add random_state for reproducibility
df_0.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
22225,ISIC_6731651,IP_3008149,male,65.0,torso,unknown,benign,0
22183,ISIC_6717322,IP_7404937,male,60.0,torso,unknown,benign,0
22135,ISIC_6704518,IP_9371649,male,65.0,torso,unknown,benign,0
21970,ISIC_6652021,IP_7291657,female,40.0,torso,unknown,benign,0
22292,ISIC_6757635,IP_8173459,female,50.0,head/neck,unknown,benign,0


In [8]:
df_train = pd.concat([df_1, df_0])
df_train = df_train.sample(frac=1, random_state=42)  # Add random_state for reproducibility
df_train


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
32519,ISIC_9821002,IP_5063756,male,75.0,lower extremity,melanoma,malignant,1
22237,ISIC_6736062,IP_5086592,male,55.0,torso,unknown,benign,0
6221,ISIC_1959939,IP_2516168,male,60.0,head/neck,melanoma,malignant,1
11075,ISIC_3416107,IP_3335629,male,60.0,lower extremity,melanoma,malignant,1
22679,ISIC_6882480,IP_4210911,female,30.0,torso,unknown,benign,0
...,...,...,...,...,...,...,...,...
22104,ISIC_6692281,IP_5906250,female,55.0,torso,unknown,benign,0
21171,ISIC_6426083,IP_4988871,female,35.0,torso,unknown,benign,0
22331,ISIC_6767860,IP_1920963,female,60.0,lower extremity,nevus,benign,0
22186,ISIC_6718077,IP_9588360,male,30.0,lower extremity,nevus,benign,0


In [9]:
def preprocessing(dataset):

    dataset['sex'].fillna("no sex", inplace = True)
    dataset['age_approx'].fillna(0, inplace = True)
    dataset['anatom_site_general_challenge'].fillna("NA", inplace = True)
    dataset = dataset.replace(to_replace = ['male'], value = 0)
    dataset = dataset.replace(to_replace = ['female'], value = 1)
    dataset = dataset.replace(to_replace = ['no sex'], value = 2)
    dataset = dataset.replace(to_replace = ['torso'], value = 0)
    dataset = dataset.replace(to_replace = ['lower extremity'], value = 1)
    dataset = dataset.replace(to_replace = ['upper extremity'], value = 2)
    dataset = dataset.replace(to_replace = ['head/neck'], value = 3)
    dataset = dataset.replace(to_replace = ['NA'], value = 4)
    dataset = dataset.replace(to_replace = ['palms/soles'], value = 5)
    dataset = dataset.replace(to_replace = ['oral/genital'], value = 6)
    
    return dataset


In [10]:
df_train = preprocessing(df_train)
df_train

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
32519,ISIC_9821002,IP_5063756,0,75.0,1,melanoma,malignant,1
22237,ISIC_6736062,IP_5086592,0,55.0,0,unknown,benign,0
6221,ISIC_1959939,IP_2516168,0,60.0,3,melanoma,malignant,1
11075,ISIC_3416107,IP_3335629,0,60.0,1,melanoma,malignant,1
22679,ISIC_6882480,IP_4210911,1,30.0,0,unknown,benign,0
...,...,...,...,...,...,...,...,...
22104,ISIC_6692281,IP_5906250,1,55.0,0,unknown,benign,0
21171,ISIC_6426083,IP_4988871,1,35.0,0,unknown,benign,0
22331,ISIC_6767860,IP_1920963,1,60.0,1,nevus,benign,0
22186,ISIC_6718077,IP_9588360,0,30.0,1,nevus,benign,0


In [11]:
df_test = preprocessing(pd.read_csv('/kaggle/input/siim-isic-melanoma-classification/test.csv'))
df_test

,image_name,patient_id,sex,age_approx,anatom_site_general_challenge
0,ISIC_0052060,IP_3579794,0,70.0,4
1,ISIC_0052349,IP_7782715,0,40.0,1
2,ISIC_0058510,IP_7960270,1,55.0,0
3,ISIC_0073313,IP_6375035,1,50.0,0
4,ISIC_0073502,IP_0589375,1,45.0,1
...,...,...,...,...,...
10977,ISIC_9992485,IP_4152479,0,40.0,0
10978,ISIC_9996992,IP_4890115,0,35.0,0
10979,ISIC_9997917,IP_2852390,0,25.0,2
10980,ISIC_9998234,IP_8861963,0,65.0,1


In [12]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if (isinstance(value, type(tf.constant(0)))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


In [13]:
def image_example(image_string, sex, age_approx,anatom_site_general_challenge, target, isTrain, image_name):

    if (isTrain):
        feature = {
            'sex': _int64_feature(sex),
            'age_approx': _float_feature(age_approx),
            'anatom_site_general_challenge': _int64_feature(anatom_site_general_challenge),
            'target': _int64_feature(target),
            'image_raw': _bytes_feature(image_string),
            }
    else:
        feature = {
            'image_name': _bytes_feature(image_name),
            'sex': _int64_feature(sex),
            'age_approx': _float_feature(age_approx),
            'anatom_site_general_challenge': _int64_feature(anatom_site_general_challenge),
            'image_raw': _bytes_feature(image_string),
            }
        
    return tf.train.Example(features=tf.train.Features(feature=feature))


In [14]:
record_file = '/kaggle/working/train.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
    for row in df_train.iterrows():
#         image_string = open('/kaggle/input/siim-isic-melanoma-classification/jpeg/train/' + row[1]['image_name'] + '.jpg', 'rb').read()
        im = cv2.imread('/kaggle/input/siim-isic-melanoma-classification/jpeg/train/' + row[1]['image_name'] + '.jpg')
        im_resize = cv2.resize(im, (300, 300))
        is_success, im_buf_arr = cv2.imencode(".jpg", im_resize)
        image_string = im_buf_arr.tobytes()
        tf_example = image_example(image_string,row[1]['sex'], row[1]['age_approx'],row[1]['anatom_site_general_challenge'], row[1]['target'], isTrain = True,image_name = None)
        writer.write(tf_example.SerializeToString())


In [15]:
# record_file = '/kaggle/working/test.tfrecords'
# with tf.io.TFRecordWriter(record_file) as writer:
#     for row in df_test.iterrows():
# #         image_string = open('/kaggle/input/siim-isic-melanoma-classification/jpeg/train/' + row[1]['image_name'] + '.jpg', 'rb').read()
#         im_test = cv2.imread('/kaggle/input/siim-isic-melanoma-classification/jpeg/test/' + row[1]['image_name'] + '.jpg')
#         im_resize = cv2.resize(im_test, (256, 256))
#         is_success, im_buf_arr = cv2.imencode(".jpg", im_resize)
#         image_string = im_buf_arr.tobytes()
#         tf_example = image_example(image_string,row[1]['sex'], row[1]['age_approx'],row[1]['anatom_site_general_challenge'], None, isTrain = False,image_name = bytes(row[1]['image_name'], 'utf-8'))
#         writer.write(tf_example.SerializeToString())

In [16]:
BATCH_SIZE = 16
SHUFFLE_SIZE = 1440
STEPS_PER_EPOCH = int(len(df_train.index)/BATCH_SIZE)

In [17]:
def _parse_function_train(proto):
    # define your tfrecord again. Remember that you saved your image as a string.
    keys_to_features = {'image_raw': tf.io.FixedLenFeature([], tf.string),
                        'target': tf.io.FixedLenFeature([], tf.int64),
                        'sex': tf.io.FixedLenFeature([], tf.int64),
                        'age_approx': tf.io.FixedLenFeature([], tf.float32),
                       'anatom_site_general_challenge': tf.io.FixedLenFeature([],tf.int64)}
    
    # Load one example
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    
    image_shape = tf.stack([300,300,3])
#     target_shape = tf.stack([1])
    
    # Turn your saved image string into an array
    parsed_features['image_raw'] = tf.io.decode_jpeg(parsed_features['image_raw'], 3)#, fixed_length = 256*256*3)
    
#     parsed_features['image'] = tf.cast(parsed_features['image']/255, tf.float32)

    
    parsed_features['image_raw'] = tf.reshape(parsed_features['image_raw'], image_shape)
    
    parsed_features['image_raw'] = tf.image.random_brightness(parsed_features['image_raw'],0.35)
    
    parsed_features['image_raw'] = tf.image.random_contrast(parsed_features['image_raw'],0.1,0.36)
    
    parsed_features['image_raw'] = tf.image.random_flip_left_right(parsed_features['image_raw'])
    
    parsed_features['image_raw'] = tf.image.random_flip_up_down(parsed_features['image_raw'])
    
    
#     parsed_features["target"] = tf.reshape(parsed_features['target'],target_shape)
    
    return parsed_features['image_raw'], parsed_features['sex'], parsed_features['age_approx'], parsed_features['anatom_site_general_challenge'],parsed_features['target']

In [18]:
def dataset_fetch (filenames, isTrain):
    
    dataset = tf.data.TFRecordDataset(filenames)
    
    dataset = dataset.repeat()

    if(isTrain):
        dataset = dataset.map(_parse_function_train)

#     else:
#         dataset = dataset.map(_parse_function_test)
        
    dataset = dataset.shuffle(SHUFFLE_SIZE, seed=42)

    dataset = dataset.batch(BATCH_SIZE)
        
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
            
    return dataset

In [19]:
# from PIL import Image
training_dataset = dataset_fetch('/kaggle/working/train.tfrecords',True)
# training_dataset = training_dataset.repeat(-1)
training_dataset


<PrefetchDataset shapes: ((None, 300, 300, 3), (None,), (None,), (None,), (None,)), types: (tf.uint8, tf.int64, tf.float32, tf.int64, tf.int64)>

In [20]:
def fetch_data(iterator):
#     image,sex,age,site,target = iterator.get_next()
    while True:
        image,sex,age,site,target = iterator.get_next()
        yield ([image,sex,age,site],target)

In [21]:
iterator = iter(training_dataset)
training_dataset_final = fetch_data(iterator)

print(training_dataset_final)

<generator object fetch_data at 0x7e26f8b97ed0>


In [22]:
image_input = tf.keras.layers.Input(shape = (300,300,3))
x1 = tf.keras.applications.Xception(weights = 'imagenet', include_top = False)(image_input)
x1 = tf.keras.layers.MaxPooling2D((2,2))(x1)
x1 = tf.keras.layers.Flatten()(x1)
x1 = tf.keras.layers.Dense(128, activation = 'relu')(x1)
x1 = tf.keras.layers.Dropout(0.2)(x1)
# x1 = tf.keras.layers.Dense(128, activation = 'relu')(x1)
# x1 = tf.keras.layers.Dropout(0.2)(x1)
out1 = tf.keras.layers.Dense(32, activation = 'relu',kernel_regularizer = tf.keras.regularizers.l2())(x1)


age_input = tf.keras.layers.Input(shape = (1))
# x2 = tf.keras.layers.Dense(128,activation = 'relu')(age_input)
x2 = tf.keras.layers.Dense(64, activation = 'relu')(age_input)
x2 = tf.keras.layers.Dropout(0.2)(x2)
out2 = tf.keras.layers.Dense(32, activation = 'relu',kernel_regularizer = tf.keras.regularizers.l2())(x2)

gender_input = tf.keras.layers.Input(shape = (1))
# x3 = tf.keras.layers.Dense(128,activation = 'relu')(gender_input)
x3 = tf.keras.layers.Dense(64, activation = 'relu')(gender_input)
x3 = tf.keras.layers.Dropout(0.2)(x3)
out3 = tf.keras.layers.Dense(32, activation = 'relu',kernel_regularizer = tf.keras.regularizers.l2())(x3)

site_input = tf.keras.layers.Input(shape = (1))
# x4 = tf.keras.layers.Dense(128,activation = 'relu')(site_input)
x4 = tf.keras.layers.Dense(64, activation = 'relu')(site_input)
x4 = tf.keras.layers.Dropout(0.2)(x4)
out4 = tf.keras.layers.Dense(32, activation = 'relu',kernel_regularizer = tf.keras.regularizers.l2())(x4)

merge1 = tf.keras.layers.concatenate([out1,out3,out2,out4])

op = tf.keras.layers.Dense(64, activation = 'relu')(merge1)
op = tf.keras.layers.Dropout(0.3)(op)
# op = tf.keras.layers.Dense(64, activation = 'relu')(op)
op = tf.keras.layers.Dense(16, activation = 'relu',kernel_regularizer = tf.keras.regularizers.l2())(op)
output_final = tf.keras.layers.Dense(1, activation = 'sigmoid')(op)

model = tf.keras.models.Model(inputs = [image_input,gender_input,age_input, site_input], outputs = output_final)


83689472/83683744 [==============================] - 4s 0us/step


In [23]:
model.compile( optimizer=tf.keras.optimizers.Adamax(),
    loss='binary_crossentropy',
    metrics=['accuracy'],)
#     target_tensors = [image_raw])

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
xception (Model)                multiple             20861480    input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 5, 5, 2048)   0           xception[1][0]                   
__________________________________________________________________________________________________
flatten (Flatten)               (None, 51200)        0           max_pooling2d[0][0]              
______________________________________________________________________________________________

In [25]:
from tensorflow.keras.callbacks import EarlyStopping

# Define EarlyStopping callback to monitor training accuracy
early_stopping = EarlyStopping(
    monitor='accuracy',    # Monitor training accuracy (instead of validation accuracy)
    patience=10,           # Stop after 10 epochs of no improvement
    restore_best_weights=True,  # Restore best model weights after stopping
    verbose=1              # Show message when early stopping is triggered
)


In [26]:
# Train the model without splitting for validation
history = model.fit(
    training_dataset_final,            # Training on the entire dataset
    epochs=50,                         # Set max epochs
    steps_per_epoch=STEPS_PER_EPOCH,   # Number of steps per epoch
    callbacks=[early_stopping]         # Add the EarlyStopping callback
)


Epoch 1/50
90/90 [==============================] - 33s 365ms/step - loss: 2.3779 - accuracy: 0.5229
Epoch 2/50
90/90 [==============================] - 33s 365ms/step - loss: 1.7347 - accuracy: 0.5361
Epoch 3/50
90/90 [==============================] - 33s 364ms/step - loss: 1.4917 - accuracy: 0.5347
Epoch 4/50
90/90 [==============================] - 33s 364ms/step - loss: 1.3589 - accuracy: 0.5625
Epoch 5/50
90/90 [==============================] - 33s 364ms/step - loss: 1.0760 - accuracy: 0.6444
Epoch 9/50
90/90 [==============================] - 33s 365ms/step - loss: 0.9770 - accuracy: 0.7229
Epoch 10/50
90/90 [==============================] - 33s 365ms/step - loss: 0.9142 - accuracy: 0.7632
Epoch 11/50
90/90 [==============================] - 33s 363ms/step - loss: 0.9077 - accuracy: 0.7507
Epoch 12/50
90/90 [==============================] - 33s 364ms/step - loss: 0.8012 - accuracy: 0.7937
Epoch 13/50
90/90 [==============================] - 33s 364ms/step - loss: 0.7522 - acc

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
gc.collect()

49

In [28]:
predictions_list = np.array([], dtype = 'float32')
print(predictions_list)
for row in df_test.iterrows():
    im_test = cv2.imread('/kaggle/input/siim-isic-melanoma-classification/jpeg/test/' + row[1]['image_name'] + '.jpg')
    im_resize = np.reshape(cv2.resize(im_test, (300, 300)), (1,300,300,3))
    predictions_list = np.append(predictions_list,model.predict([im_resize, np.reshape(row[1]['sex'],(1)), 
                                                                                np.reshape(row[1]['age_approx'],(1)), 
                                                                                    np.reshape(row[1]['anatom_site_general_challenge'],(1))])[0])#[1])

    print(predictions_list.shape, end = "\r")


[]


In [29]:
gc.collect()

73023

In [30]:
unique_elements, counts_elements = np.unique(predictions_list, return_counts=True)

print(np.asarray((unique_elements, counts_elements)))

[[2.96393431e-11 3.07462944e-11 4.03796406e-11 ... 9.99999762e-01
  9.99999881e-01 1.00000000e+00]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.00000000e+00
  3.00000000e+00 1.00000000e+00]]


In [31]:
test_image_name = df_test['image_name'].to_numpy()

In [32]:
sample_submission = pd.DataFrame({"image_name":test_image_name, "target":predictions_list})
sample_submission

,image_name,target
0,ISIC_0052060,0.534816
1,ISIC_0052349,0.000205
2,ISIC_0058510,0.000013
3,ISIC_0073313,0.001231
4,ISIC_0073502,0.000012
...,...,...
10977,ISIC_9992485,0.022647
10978,ISIC_9996992,0.994497
10979,ISIC_9997917,0.999822
10980,ISIC_9998234,0.741512


In [33]:
sample_submission.to_csv("submission.csv",index = False)

In [34]:
# Save the entire model
model.save('/kaggle/working/model_with_out_splitting_1.h5')  # Save the model as an HDF5 file (or you can specify other file paths)

print("Model saved successfully!")

Model saved successfully!
